# Model Training
Welcome to the 'Model Training and Prediction' notebook, a crucial facet of our project's data science pipeline. In this notebook, we offer a meticulous examination of our rigorous model development process. The pipeline starts by accepting training data, followed by fitting three distinct types of models to it: Random Forest, Gradient Boosted Tree, and XGBoost. The initial stages include encoding categorical variables and executing Recursive Feature Elimination (RFE) for feature selection. This is succeeded by the application of genetic algorithms to hyperparameter tuning, operating in tandem with a cross-validation routine. Subsequently, the best model is selected based on the highest F1 score, indicating the balance between precision and recall. Finally, the selected model is utilized to predict the outcomes for the current week's round of NRL matches. This process is iterative and cyclical, with the potential for revisiting earlier stages based on the model's performance. Let us proceed with this in-depth exploration.

## Set up Environment
This code segment is setting up the environment for the model training pipeline. It begins by importing sys and pathlib - Python libraries used for managing system parameters and file paths, respectively.

The code then updates the system path to include the "functions" directory. This allows for the import of custom modules `modelling_functions`, `model_properties`, and `training_config` which are stored in this directory. These modules contain custom functions and configuration settings that are critical for the later stages of data preprocessing, model training, and prediction.

Following this, the `project_root` variable is defined. This is achieved by using the pathlib library to establish the root directory of the project.

Finally, the `db_path` variable is constructed. This is the relative path to the SQLite database "footy-tipper-db.sqlite", which is located in the "data" directory of the project root. This path will be used for database connectivity throughout the pipeline.

In [1]:
import sys
import pathlib

sys.path.append("functions") 
import modelling_functions as mf
import model_properties as mp
import training_config as tc
import prediction_functions as pf

# Get to the root directory
project_root = pathlib.Path().absolute().parent.parent

# Now construct the relative path to your SQLite database
db_path = project_root / "data" / "footy-tipper-db.sqlite"

## Get data
Our process starts by establishing the root directory of the project and constructing the relative path to the 'footy-tipper-db.sqlite' database located within the 'data' directory. We then connect to this SQLite database and use a SQL query housed in the 'footy_tipping_data.sql' file, found in the 'sql' directory, to extract the required data. This data is loaded into a pandas DataFrame, footy_tipping_data, serving as the basis for our subsequent modeling activities. Upon successful extraction of the data, we ensure the database connection is closed, maintaining good coding practice and resource management.

In [2]:
training_data = mf.get_training_data(db_path, 'sql/training_data.sql')
training_data

,game_id,round_id,round_name,game_number,game_state_name,start_time,start_time_utc,venue_name,city,crowd,...,away_prev_result_diff,prev_result_diff,home_elo,away_elo,elo_diff,home_elo_prob,away_elo_prob,elo_draw_prob,elo_prob_diff,home_ground_advantage
0,2.020111e+10,1.0,Round 1,1.0,Final,1.584044e+09,1.584004e+09,CommBank Stadium,Sydney,21363.0,...,0.0,0.0,1510.549824,1496.564196,13.985628,0.386836,0.352120,0.261044,0.034717,9.231356
1,2.020111e+10,1.0,Round 1,2.0,Final,1.584122e+09,1.584083e+09,GIO Stadium,Canberra,10610.0,...,0.0,0.0,1515.703583,1466.531363,49.172220,0.569782,0.423551,0.006667,0.146231,8.205272
2,2.020111e+10,1.0,Round 1,3.0,Final,1.584126e+09,1.584090e+09,Queensland Country Bank Stadium,Townsville,22459.0,...,0.0,0.0,1490.398541,1483.395645,7.002897,0.379421,0.359535,0.261044,0.019886,1.395793
3,2.020111e+10,1.0,Round 1,4.0,Final,1.584198e+09,1.584158e+09,McDonald Jones Stadium,Newcastle,10239.0,...,0.0,0.0,1484.826777,1486.536417,-1.709640,0.370158,0.368798,0.261044,0.001360,7.771252
4,2.020111e+10,1.0,Round 1,5.0,Final,1.584207e+09,1.584167e+09,Accor Stadium,Sydney,NaN,...,0.0,0.0,1505.183636,1506.312184,-1.128548,0.370776,0.368180,0.261044,0.002596,2.379140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,2.023111e+10,27.0,Round 27,4.0,Final,1.693667e+09,1.693631e+09,Suncorp Stadium,Brisbane,35438.0,...,-8.0,-16.0,1463.177763,1529.508774,-66.331011,0.408950,0.591050,0.000000,-0.182101,-2.827330
771,2.023111e+10,27.0,Round 27,5.0,Final,1.693676e+09,1.693640e+09,BlueBet Stadium,Penrith,21525.0,...,-24.0,10.0,1548.509706,1521.064727,27.444979,0.539154,0.454179,0.006667,0.084974,11.027407
772,2.023111e+10,27.0,Round 27,6.0,Final,1.693683e+09,1.693647e+09,Netstrata Jubilee Stadium,Sydney,10137.0,...,-2.0,-8.0,1467.507645,1545.428119,-77.920475,0.392927,0.607073,0.000000,-0.214145,4.806468
773,2.023111e+10,27.0,Round 27,7.0,Final,1.693750e+09,1.693714e+09,Cbus Super Stadium,Gold Coast,14942.0,...,12.0,-38.0,1473.273079,1428.976390,44.296689,0.562950,0.430383,0.006667,0.132567,13.898976


## Modelling
During the modelling phase, the `train_and_select_best_model` function, part of our `modelling_functions` module, is invoked. This function initiates the training of three distinct models: XGBoost, Random Forest, and Gradient Boosting Classifier. It takes as input the footy tipping data, predictor variables, the outcome variable, and several configuration settings like whether to use Recursive Feature Elimination (RFE), the number of cross-validation folds, and the optimization metric, all sourced from the `training_config` module.

The function first identifies categorical columns in the feature set for one-hot encoding, creating dummy variables for categorical features. Depending on the choice of using RFE, a feature elimination step may be included in the pipeline. Each model subsequently undergoes hyperparameter tuning using a genetic algorithm, facilitated by the `GASearchCV` function.

All the models are then trained and evaluated through cross-validation. The best model, or `footy_tipper`, is selected based on the superior performance on the chosen optimization metric. Additionally, a `LabelEncoder`(`label_encoder`), used to encode the categorical target variable, is returned. This LabelEncoder is specific to the model that performed best. The selected model, encapsulated in a pipeline with pre-processing steps and hyperparameter tuning, is now ready for the prediction phase.

In [3]:
footy_tipper, label_encoder = mf.train_and_select_best_model(
    training_data, tc.predictors, tc.outcome_var,
    tc.use_rfe, tc.num_folds, tc.opt_metric
)

footy_tipper


Model training: XGBClassifier
gen	nevals	fitness	fitness_std	fitness_max	fitness_min
0  	100   	0.71191	0.0120641  	0.730323   	0.665806   
1  	138   	0.720813	0.00614675 	0.734194   	0.701935   
2  	151   	0.725961	0.00439763 	0.739355   	0.709677   
3  	145   	0.729058	0.00371729 	0.738065   	0.72129    
4  	144   	0.731458	0.00314627 	0.738065   	0.723871   
5  	137   	0.733613	0.00265929 	0.738065   	0.725161   
6  	138   	0.735174	0.0021368  	0.739355   	0.729032   
7  	139   	0.736465	0.00192358 	0.739355   	0.727742   
8  	149   	0.737381	0.00130896 	0.739355   	0.734194   
9  	144   	0.737974	0.000917037	0.739355   	0.732903   
10 	131   	0.738323	0.000605215	0.739355   	0.735484   
11 	131   	0.738387	0.0012292  	0.739355   	0.730323   
12 	136   	0.738555	0.00127448 	0.739355   	0.730323   
13 	145   	0.738839	0.0011541  	0.739355   	0.730323   
14 	133   	0.738981	0.00150802 	0.739355   	0.730323   
15 	136   	0.739123	0.00186433 	0.739355   	0.72129    
16 	152   	0.739239

Pipeline(steps=[('one_hot_encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['round_name', 'venue_name',
                                                   'city', 'team_home',
                                                   'team_away',
                                                   'broadcast_channel1',
                                                   'broadcast_channel2',
                                                   'broadcast_channel3',
                                                   'game_day'])])),
                ('feature_elimination',
                 RFECV(cv=5,
                       estimator=XGBClassifier(base_score=Non...
                                        'learning_rate': <sklearn_genetic.space.space.Continuous object at 0x16c894f90>,
                                        'max_depth': <sklearn_genetic.space.space.Integer object at 0x16c897990>,
                                        'n_estimators': <sklearn_genetic.space.space.Integer object at 0x12ee2fe50>,
                                        'subsample': <sklearn_genetic.space.space.Continuous object at 0x16c897f10>},
                            population_size=100, return_train_score=True,
                            scoring='accuracy'))])

### Display feature importance
The `get_feature_importance` function retrieves feature importances from a trained scikit-learn pipeline. It accounts for different transformations, such as one-hot encoding and recursive feature elimination. The function then returns a sorted DataFrame listing each feature alongside its respective importance, aiding in understanding the model's decision-making process.

In [4]:
# feature_importance_df = mp.get_feature_importances_from_pipeline(footy_tipper, tc.predictors)
# feature_importance_df

## Save Model
The `save_models` function stores the trained LabelEncoder and Pipeline objects to the disk. This allows for easy retrieval and reuse in future model prediction tasks, without the need to retrain these components. The objects are stored in a designated 'models' directory under the project root path, ensuring organized and consistent storage.

In [5]:
# mf.save_models(label_encoder, footy_tipper, project_root)

In [6]:
inference_data = pf.get_inference_data(db_path, 'sql/inference_data.sql')
inference_data

,game_id,round_id,round_name,game_number,game_state_name,start_time,start_time_utc,venue_name,city,crowd,...,away_prev_result_diff,prev_result_diff,home_elo,away_elo,elo_diff,home_elo_prob,away_elo_prob,elo_draw_prob,elo_prob_diff,home_ground_advantage
0,2.023111e+10,28.0,Finals Week 1,1.0,Pre Game,1.694203e+09,1.694167e+09,Suncorp Stadium,Brisbane,None,...,-10.0,0.0,1542.070817,1548.658479,-6.587662,0.364971,0.373985,0.261044,-0.009015,-1.509433
1,2.023111e+10,28.0,Finals Week 1,2.0,Pre Game,1.694276e+09,1.694240e+09,BlueBet Stadium,Penrith,None,...,24.0,8.0,1559.515643,1512.635244,46.880400,0.566574,0.426759,0.006667,0.139815,13.568667
2,2.023111e+10,28.0,Finals Week 1,3.0,Pre Game,1.694289e+09,1.694253e+09,PointsBet Stadium,Sydney,None,...,-14.0,32.0,1520.301167,1513.177647,7.123519,0.379549,0.359407,0.261044,0.020142,-1.240133
3,2.023111e+10,28.0,Finals Week 1,4.0,Pre Game,1.694362e+09,1.694326e+09,McDonald Jones Stadium,Newcastle,None,...,18.0,8.0,1546.683032,1470.037556,76.645476,0.607842,0.385748,0.006410,0.222093,7.354400


In [7]:
predictions_df = pf.model_predictions(footy_tipper, inference_data, label_encoder)
predictions_df

,game_id,home_team_result,home_team_win_prob,home_team_lose_prob
0,2.023111e+10,Win,0.537712,0.462288
1,2.023111e+10,Win,0.920733,0.079267
2,2.023111e+10,Loss,0.497588,0.502412
3,2.023111e+10,Win,0.852072,0.147928


In [8]:
pf.save_predictions_to_db(predictions_df, db_path, 'sql/create_table.sql', 'sql/insert_into_table.sql')